In [1]:
import pandas as pd
import numpy as np

In [2]:
labels = ["s", "d", "b", "f", "q"]
label2id = {label:idx for idx, label in enumerate(labels)}

In [3]:
# load in session 16 of the BERT classified transcripts dialog
trans = pd.read_csv('results/trans_results.csv')
s16 = trans[trans['session'] == 2116]

In [4]:
from sklearn.preprocessing import OneHotEncoder

# preproccess BERT machine labeled labels by putting them into one-hot format
onehot_enc = OneHotEncoder(sparse=False)
bert_labels_raw = s16['labels'][1:].to_numpy().reshape(s16['labels'].size-1,1)
bert_labels = onehot_enc.fit_transform(bert_labels_raw)

/home/mms9355/miniconda3/envs/venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.

In [ ]:
# milans labels (human #1)

# transform labels to one-hot format
h1_df_raw=pd.read_csv('fewshot_labels/trans16_milan.csv')
h1_raw = h1_df_raw.replace({'label': label2id}).to_numpy()
h1_labels = onehot_enc.fit_transform(h1_raw)

/home/mms9355/miniconda3/envs/venv/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# calculate fleiss's kappa
# equation lifted from here: https://en.wikipedia.org/wiki/Fleiss%27_kappa
def fleiss_kappa(Z, n):
    N = Z.shape[0]

    p_j = np.apply_along_axis(np.sum,0, Z)/(n*N)
    p_i = 1/(n * (n-1)) * np.sum(np.hstack((np.square(Z), np.full(N,-n).reshape(N,1))),1) 

    p_bar = 1/N * np.sum(p_i)
    p_e = np.sum(np.square(p_j))
    kappa = (p_bar - p_e)/(1-p_e)
    return kappa

In [ ]:
# an array of all the raters
all_labeled = np.array([h1_labels, bert_labels])

# calculate frequency matrix used in one-hot calculation
Z = np.sum(all_labeled, 0)
n = len(all_labeled)

# calculate the kappa value
fleiss_kappa(Z,n)

0.4987064511506174